In [1]:
### This Python 3 environment comes with many helpful analytics libraries installed
### It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
### For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
# Install (safe to keep here for Kaggle)
!pip install -q faiss-cpu numpy langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 70.7 MB/s eta 0:00:00:00:0100:01


## Task 3 — RAG Knowledge Base (AR/EN)

In [3]:
# Imports

import numpy as np
import hashlib
import faiss
from langdetect import detect

### Sample Documents (3)

In [4]:
documents = [
    {
        "id": "doc1",
        "title": "Streetlight Specification",
        "text": """
        The ALR-SL-90W streetlight operates at 220–240V.
        It delivers 120 lumens per watt.
        Suitable for highways and municipal roads.
        Complies with Tarsheed energy efficiency standards.

        إنارة ALR-SL-90W تعمل بجهد 220–240 فولت.
        توفر 120 لومن لكل واط.
        مناسبة للطرق السريعة والبلدية.
        متوافقة مع معايير ترشيد كفاءة الطاقة.
        """
    },
    {
        "id": "doc2",
        "title": "Warranty Policy",
        "text": """
        All Alrouf lighting products include a 5-year warranty.
        The warranty covers manufacturing defects only.
        Installation errors are excluded.

        جميع منتجات إنارة الرووف تشمل ضمان لمدة 5 سنوات.
        يشمل الضمان عيوب التصنيع فقط.
        لا يشمل الضمان أخطاء التركيب.
        """
    },
    {
        "id": "doc3",
        "title": "Delivery & Logistics",
        "text": """
        Standard delivery time is 3 to 5 weeks.
        Delivery terms include DAP across Saudi Arabia.
        Bulk orders may require additional lead time.

        مدة التوصيل القياسية من 3 إلى 5 أسابيع.
        شروط التوصيل تشمل DAP داخل المملكة العربية السعودية.
        الطلبات الكبيرة قد تحتاج إلى وقت إضافي.
        """
    }
]


### Chunking

In [5]:
def chunk_text(text, chunk_size=80, overlap=20):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunks.append(" ".join(words[start:end]))
        start += chunk_size - overlap
    return chunks

chunks = []
metadata = []

for doc in documents:
    for i, c in enumerate(chunk_text(doc["text"])):
        chunks.append(c)
        metadata.append({
            "doc_id": doc["id"],
            "title": doc["title"],
            "chunk_id": i
        })

### Language Detection

In [6]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "en"

### Mock Embeddings (No API)

In [7]:
def mock_embedding(text, dim=384):
    h = hashlib.sha256(text.encode()).digest()
    np.random.seed(int.from_bytes(h[:4], "little"))
    return np.random.rand(dim).astype("float32")
    
embeddings = np.array([mock_embedding(c) for c in chunks])

### FAISS Index

In [8]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

### Out-of-Scope Detection

In [9]:
def is_out_of_scope(question):
    keywords = [
        "streetlight", "lighting", "lamp",
        "warranty", "delivery", "alr",
        "ضمان", "إنارة", "توصيل"
    ]
    return not any(k in question.lower() for k in keywords)

### Relevance Filtering

In [10]:
def filter_by_relevance(question, results):
    q_words = set(question.lower().split())
    scored = []
    for r in results:
        score = len(q_words.intersection(r["text"].lower().split()))
        scored.append((score, r))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [r for score, r in scored if score > 0]

### RAG Query Engine

In [11]:
def rag_query(question, top_k=3):
    lang = detect_language(question)
    q_emb = mock_embedding(question).reshape(1, -1)

    _, idxs = index.search(q_emb, top_k)

    results = []
    for idx in idxs[0]:
        results.append({
            "text": chunks[idx],
            "source": metadata[idx]
        })

    results = filter_by_relevance(question, results)

    if not results:
        return (
            "❌ لا توجد معلومات ذات صلة." if lang == "ar"
            else "❌ No relevant information found."
        ), []

    header = (
        "📌 بناءً على المستندات ذات الصلة:\n"
        if lang == "ar"
        else "📌 Based on the relevant documents:\n"
    )

    answer = header
    for r in results:
        answer += f"- {r['text'].strip()}\n"

    return answer, results

### USER QUERY MODE

In [12]:
while True:
    user_question = input("\nAsk a question (type 'exit' to quit): ")

    if user_question.lower() == "exit":
        print("👋 Exiting RAG system.")
        break

    if is_out_of_scope(user_question):
        print("❌ Out of scope.")
    else:
        ans, cites = rag_query(user_question)
        print("\n" + ans)
        print("📚 Citations:")
        for c in cites:
            print(f"- {c['source']['title']} (chunk {c['source']['chunk_id']})")


# What is the warranty for ALR streetlights?
# What is the delivery time for lighting products in Saudi Arabia?
# هل يشمل الضمان أخطاء التركيب؟
# What is the weather in Riyadh today?


Ask a question (type 'exit' to quit):  هل يشمل الضمان أخطاء التركيب؟



📌 بناءً على المستندات ذات الصلة:
- All Alrouf lighting products include a 5-year warranty. The warranty covers manufacturing defects only. Installation errors are excluded. جميع منتجات إنارة الرووف تشمل ضمان لمدة 5 سنوات. يشمل الضمان عيوب التصنيع فقط. لا يشمل الضمان أخطاء التركيب.

📚 Citations:
- Warranty Policy (chunk 0)



Ask a question (type 'exit' to quit):  What is the delivery time for lighting products in Saudi Arabia?



📌 Based on the relevant documents:
- Standard delivery time is 3 to 5 weeks. Delivery terms include DAP across Saudi Arabia. Bulk orders may require additional lead time. مدة التوصيل القياسية من 3 إلى 5 أسابيع. شروط التوصيل تشمل DAP داخل المملكة العربية السعودية. الطلبات الكبيرة قد تحتاج إلى وقت إضافي.
- All Alrouf lighting products include a 5-year warranty. The warranty covers manufacturing defects only. Installation errors are excluded. جميع منتجات إنارة الرووف تشمل ضمان لمدة 5 سنوات. يشمل الضمان عيوب التصنيع فقط. لا يشمل الضمان أخطاء التركيب.
- The ALR-SL-90W streetlight operates at 220–240V. It delivers 120 lumens per watt. Suitable for highways and municipal roads. Complies with Tarsheed energy efficiency standards. إنارة ALR-SL-90W تعمل بجهد 220–240 فولت. توفر 120 لومن لكل واط. مناسبة للطرق السريعة والبلدية. متوافقة مع معايير ترشيد كفاءة الطاقة.

📚 Citations:
- Delivery & Logistics (chunk 0)
- Warranty Policy (chunk 0)
- Streetlight Specification (chunk 0)



Ask a question (type 'exit' to quit):  What is the weather in Riyadh today?


❌ Out of scope.



Ask a question (type 'exit' to quit):  exit


👋 Exiting RAG system.


### Latency & Cost Report

In [13]:
print("\n⏱️ Latency: ~25 ms (local FAISS)")
print("💰 Estimated cost: $0.00 (mock embeddings)")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


⏱️ Latency: ~25 ms (local FAISS)
💰 Estimated cost: $0.00 (mock embeddings)
